In [2]:
try:
    import binutil
except ModuleNotFoundError:
    import bin.binutil

from dreamcoder.program import *
from dreamcoder.domains.relation import *
from dreamcoder.domains.relation.relation_primitives import *

get_baseline_primitives()
get_clevr_primitives()

same_shape_program = Program.parse(
    "(lambda (#(lambda (#(lambda (lambda (eq? ($0 (car $1)) ($0 (car (cdr $1)))))) $0 (lambda (#(lambda (lambda (lambda (index (find $1 $2) (cdr $0))))) $0 2001 $0)))) $0))"
)

In [98]:
# read txt file
mode = "test"
split = "all_metal_one_gray_10"  # "all_cubes_8"  "all_cubes_5"
same_support = True
seed = 1

domain = "clevr"
data_folder = "curi_dc_test_tasks"

file = f"../consoleOutputs/{domain}/all_metal_one_gray/eval/{seed}/{mode}_programs.out"

with open(file, "r") as f:
    data = f.read()

if mode == "train":
    data = data.split("\n")
    data = [line for line in data if line != ""]
    task_programs = {}
    task_flag = False
    for line in data:
        if task_flag:
            program = line.split("\t")[1]
            task_programs[task_id] = program
            task_flag = False
        if "task" in line:
            task_flag = True
            task_id = line
else:
    data = data.split("\n")
    NUMBER_TEST_TASKS = len(data)
    print("Number of all tasks: ", len(data))
    task_programs = {}
    for task in data:
        if not "HIT" in task:
            continue
        task = task.split("w/")
        name = task[0].replace(" ", "")
        name = name.replace("HIT", "")
        program = task[1].split(";")[0][1:]
        task_programs[name] = program

Number of all tasks:  1


In [99]:
task_programs

{'task_4919': '(lambda (and (not (#(lambda (exists (lambda (#(lambda (lambda (eq? $0 (index 7 $1)))) $0 0)) $0)) $0)) (not (#(lambda (lambda (lambda (fold (cdr $2) (#(lambda (lambda (#(#(#(lambda (lambda (forall (lambda (eq? $1 (index $2 $0)))))) 5) 2) (if (#(#(lambda (lambda (forall (lambda (eq? $1 (index $2 $0)))))) 6 0) (#(lambda (cons (car $0) empty)) $1)) (cdr $1) $0)))) $2 $1) (lambda (lambda (#(lambda (#(lambda (lambda (lambda (and $2 (not (or (#(#(lambda (lambda (forall (lambda (eq? $1 (index $2 $0)))))) 6 1) $0) ($1 (#(lambda (cons (car $0) empty)) $0)))))))) (not (#(#(lambda (lambda (forall (lambda (eq? $1 (index $2 $0)))))) 6 0) $0)))) $4 (lambda $1) $2))))))) $0 $0 (#(lambda (cons (car $0) empty)) (if (#(lambda (#(lambda (lambda (eq? $0 (index 7 $1)))) $0 (index 3 (cdr $0)))) (car $0)) $0 (cdr $0))))))) '}

In [100]:
import pandas as pd
import json
import os
from tqdm import tqdm

# parse inputs from model_results_eval

path = f"../data/{data_folder}/{split}/query/"
json_files = [f.path for f in os.scandir(path) if f.path.endswith(".json")]

task_results = []

# iterate over tasks
for task_file in tqdm(json_files):
    task_name = task_file.split("/")[-1][:-5]
    # task_number = task_name.split("_")[-1]
    # check if program exists for task
    if task_name in task_programs.keys():
        f = open(task_file)
        examples = json.load(f)

        parsed_examples = []

        true_positives = 0
        false_positives = 0
        true_negatives = 0
        false_negatives = 0

        # print(len(examples))

        for example in examples:
            input = example["input"]
            output = example["output"]

            # execute program for input
            program = task_programs[task_name]
            program = Program.parse(program)
            try:
                program_output = program.evaluate([])(input)
            except:
                print("some issue with program ", program)
                continue

            # categorize prediction
            if output:
                if program_output:
                    true_positives += 1
                else:
                    false_negatives += 1
            else:
                if program_output:
                    false_positives += 1
                else:
                    true_negatives += 1

        # collect results
        results = {
            "task_name": task_name,
            "program": task_programs[task_name],
            "TP": true_positives,
            "FP": false_positives,
            "TN": true_negatives,
            "FN": false_negatives,
        }
        task_results.append(results)
        # print(results)
    else:
        continue
        results = {"task_name": task_name, "TP": 0, "FP": 0, "TN": 0, "FN": 0}
        task_results.append(results)

results_df = pd.DataFrame(task_results)
results_df

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:05<00:00,  5.99s/it]


,task_name,program,TP,FP,TN,FN
0,task_4919,(lambda (and (not (#(lambda (exists (lambda (#...,35,0,100,65


In [101]:
results_df

,task_name,program,TP,FP,TN,FN
0,task_4919,(lambda (and (not (#(lambda (exists (lambda (#...,35,0,100,65


In [102]:
results_df["Accuracy"] = (results_df["TP"] + results_df["TN"]) / (
    results_df["TP"] + results_df["TN"] + results_df["FP"] + results_df["FN"]
)
results_df["CBA"] = (
    (results_df["TP"] / (results_df["TP"] + results_df["FN"]))
    + (results_df["TN"] / (results_df["TN"] + results_df["FP"]))
) / 2
results_df["Precision"] = results_df["TP"] / (results_df["TP"] + results_df["FP"])
# get number of tasks with accuracy not nan
len(results_df[~results_df["Accuracy"].isna()])

1

In [103]:
results_df.sort_values(by="CBA", ascending=True)

,task_name,program,TP,FP,TN,FN,Accuracy,CBA,Precision
0,task_4919,(lambda (and (not (#(lambda (exists (lambda (#...,35,0,100,65,0.675,0.675,1.0


In [104]:
print(split, " ", mode)
# get mean accuracy
print(results_df["Accuracy"].mean())
# get mean CBA
print(results_df["CBA"].mean())
# get precision
print(results_df["Precision"].mean())

all_metal_one_gray_10   test
0.675
0.675
1.0


In [105]:
print(split, " ", mode, " ", seed)
print("Class balanced accuracy (solved): ", results_df["CBA"].mean())

cba_50 = results_df["CBA"].mean() * (len(results_df) / NUMBER_TEST_TASKS) + 0.5 * (
    1 - (len(results_df) / NUMBER_TEST_TASKS)
)
# round to 2 decimal places
cba_50 = round(cba_50 * 100, 2)
print("Class balanced accuracy (all): ", cba_50)

all_metal_one_gray_10   test   1
Class balanced accuracy (solved):  0.675
Class balanced accuracy (all):  67.5


In [97]:
# save results
results_df.to_csv(f"../experimentOutputs/{domain}/{split}_{mode}.csv")
print("saved results to ", f"../experimentOutputs/{domain}/{split}_{mode}.csv")

saved results to  ../experimentOutputs/clevr/all_metal_one_gray_8_test.csv


In [109]:
import numpy as np

# accs = [100, 67, 100]
accs = [100, 67.5, 100]
mean = np.mean(accs)
std = np.std(accs)
print("mean: ", mean)
print("std: ", std)

print(round(mean, 2), "$\pm$", round(std, 2))

mean:  89.16666666666667
std:  15.320646925708528
89.17 $\pm$ 15.32
